In [15]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *

import tensorflow as tf
import tensorflow_hub as thub
import bert

import pandas as pd

import re

import random

import os
from tqdm import tqdm

import model_utils

In [16]:
bert_layer, tokenizer = model_utils.init_bert()

In [17]:
sc, spark = model_utils.init_spark()

In [20]:
sarcastic, non_sarcastic, ratio = model_utils.load_data(spark, 
                                                        bucket_name="sarc-bucket-5", 
                                                        dataset="reddit_processed")

TypeError: load_data() got multiple values for argument 'bucket_name'

In [8]:
spark

In [14]:
sc.stop()